<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install imageio

In [1]:
import imageio
import os


image_directory = '/content/drive/MyDrive/NTU_DLCV/p1_data/train_50'
image_files = [file for file in os.listdir(image_directory) if file.endswith('.png')]

In [3]:
from concurrent.futures import ProcessPoolExecutor


x_train = []
def read_image(filename):
    image_path = os.path.join(image_directory, filename)
    return imageio.imread(image_path)

# 使用多進程並行處理以加快讀取速度
with ProcessPoolExecutor() as executor:
    x_train = list(executor.map(read_image, image_files))

y_train = [int(classification.split('_')[0]) for classification in image_files]

<ipython-input-3-3b24ed707ec1>:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(image_path)
<ipython-input-3-3b24ed707ec1>:8: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(image_path)


KeyboardInterrupt: ignored

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MyDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.data = data
        self.targets = targets
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index], self.targets[index]

        if self.transform:
            sample = self.transform(sample)

        return sample

# 定義資料轉換
preprocess = transforms.Compose([
    transforms.ToTensor(), # Convert to Tensor
    # ...
])
# 創建自定義 Dataset
hw1_dataset = MyDataset(x_train, y_train, transform=preprocess)

In [52]:
image_directory = '/content/drive/MyDrive/NTU_DLCV/p1_data/val_50'
image_files = [file for file in os.listdir(image_directory) if file.endswith('.png')]

x_val = []
def read_image(filename):
    image_path = os.path.join(image_directory, filename)
    return imageio.imread(image_path)

# 使用多進程並行處理以加快讀取速度
with ProcessPoolExecutor() as executor:
    x_val = list(executor.map(read_image, image_files))
x_val = torch.Tensor(x_val)
x_val_norm = x_val / 255.0
x_val_norm = x_val_norm.permute(0, 3, 1, 2)

y_val = [int(classification.split('_')[0]) for classification in image_files]
y_val = torch.Tensor(y_val)
y_val = y_val.to(torch.int)

<ipython-input-52-66f948d01f71>:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(image_path)
<ipython-input-52-66f948d01f71>:7: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return imageio.imread(image_path)


In [53]:
# Use DataLoader to generate minibatches
BATCH_SIZE = 250
x_train_dataloader = torch.utils.data.DataLoader(
    x_train_norm,
    batch_size=BATCH_SIZE,
    shuffle=True)

x_val_dataloader = torch.utils.data.DataLoader(
    x_train_norm,
    batch_size=BATCH_SIZE)